<a href="https://colab.research.google.com/github/mafaldasalomao/DCGAN_MNIST_TENSORFLOW/blob/main/cGAN_pretrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2 # opencv
import os
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow

In [ ]:
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

In [ ]:
%cd pytorch-CycleGAN-and-pix2pix/

In [ ]:
!pip install -r requirements.txt

## Baixar e carregar o modelo pré-treinado

Você poderia também baixar os modelos pré-treinados aqui: https://github.com/affinelayer/pix2pix-tensorflow#datasets-and-trained-models

> Link com os modelos: http://efrosgans.eecs.berkeley.edu/pix2pix/models-pytorch/

Modelos disponíveis: `edges2shoes` (desenhos->sapatos), `sat2map` (satélite->mapa), `map2sat` (mapa->satélite), `facades_label2photo` (label de fachadas ->fotos de fachadas), `day2night` (dia->noite) e `edges2handbags` (desenhos->bolsas)

In [ ]:
nome_modelo = 'facades_label2photo'

In [ ]:
diretorio_modelo = "./checkpoints/{}_pretreinado".format(nome_modelo)


In [ ]:
!mkdir -p {diretorio_modelo}

In [ ]:
url_modelo = 'http://efrosgans.eecs.berkeley.edu/pix2pix/models-pytorch/{}.pth'.format(nome_modelo)
url_modelo

In [ ]:
!wget -N {url_modelo}

In [ ]:
!mv {nome_modelo}.pth {diretorio_modelo}latest_net_G.pth

## Download do dataset

Podemos baixar os datasets a partir do link: http://efrosgans.eecs.berkeley.edu/pix2pix/datasets/

Lembrando que temos as seguintes opções:
* `cityscapes`, `night2day`, `edges2handbags`, `edges2shoes`, `facades`, `maps`

In [ ]:
dataset = 'facades'
arquivo_dataset = "{}.tar.gz".format(dataset)
url_dataset = "http://efrosgans.eecs.berkeley.edu/pix2pix/datasets/{}".format(arquivo_dataset)
url_dataset

In [ ]:
!wget -N {url_dataset}

In [ ]:
# https://man7.org/linux/man-pages/man1/tar.1.html
!tar -zxvf {arquivo_dataset} -C ./datasets/

In [ ]:
## Testando o modelo

- O algoritmo vai salvar 3 imagens para cada uma das amostras de teste: 
* `*_real_A` - a imagem de entrada, (antes da tradução) 
* `*_real_B` - a imagem "real" (modelo de referência)
* `*_fake_B` - a imagem de saída (a gerada pela GAN)

In [ ]:
dataset, nome_modelo

In [ ]:
!python test.py --dataroot ./datasets/{dataset} --direction BtoA --model pix2pix --name {nome_modelo}_pretreinado

In [ ]:
diretorio_resultados = "./results/{}_pretreinado/test_latest/images/".format(nome_modelo)

In [ ]:
img_resultado = cv2.imread(diretorio_resultados + "100_fake_B.png")
cv2_imshow(img_resultado)

In [ ]:
img_resultado = cv2.imread(diretorio_resultados + "100_real_A.png")
cv2_imshow(img_resultado)

In [ ]:
img_resultado = cv2.imread(diretorio_resultados + "100_real_B.png")
cv2_imshow(img_resultado)

# Desenho para Foto


In [ ]:
nome_modelo = 'edges2shoes'
diretorio_modelo = "./checkpoints/{}_pretreinado/".format(nome_modelo)
!mkdir -p {diretorio_modelo}

In [ ]:
url_modelo = 'http://efrosgans.eecs.berkeley.edu/pix2pix/models-pytorch/{}.pth'.format(nome_modelo)
!wget -N {url_modelo}

In [ ]:
!mv {nome_modelo}.pth {diretorio_modelo}latest_net_G.pth

In [ ]:
!mkdir -p imagens/test/

## Geração dos resultados

Parâmetros: 

* ` --dataset_mode single` - define o modo como `single` o que significa que serão imagens únicas

* `--model test` - antes o valor de `--model` era pix2pix (pois seu algoritmo é preparado para trabalhar com os datasets pareados) mas agora como estamos lidando com imagem única vamos deixar como `test`, que ainda assim irá ser Pix2Pix mas agora estamos dizendo que não queremos que ele faça o processo que fez quando era imagem em par. 

* `--netG unet_256` - especifica a rede do gerador. Antes não precisávamos dizer, mas agora que fizemos tal ajuste a gente precisa dessa alteração para indicar explicitamente que vamos usar essa arquitetura padrão.

* `--norm batch` - o mesmo caso do acima, precisamos agora passar esse parâmetro mesmo já sendo o valor "padrão". 

* Obs: e repare que o parâmetro `--direction` não foi usado também, pois não precisa invertar a direção (na verdade, nesse caso não tem direção nenhuma pois definimos que é uma imagem única, diferente do outro exemplo que foi executado em um dataset pareado)   

In [ ]:
!python test.py --dataroot ./imagens --name {nome_modelo}_pretreinado --model test --dataset_mode single --netG unet_256 --norm batch

In [ ]:
diretorio_resultados = "./results/{}_pretreinado/test_latest/images/".format(nome_modelo)

In [ ]:
def exibir_resultados(nome_img, diretorio):
  img_entrada = cv2.imread(diretorio + nome_img + "_real.png")
  img_resultado = cv2.imread(diretorio + nome_img + "_fake.png")
  imgs = [img_entrada, img_resultado]
  titulo = ['Desenho (entrada)', 'Foto (saída)']
  for i in range(len(imgs)):
    plt.subplot(1,2, i + 1)
    plt.title(titulo[i])
    plt.axis('off')
    plt.imshow(cv2.cvtColor(imgs[i], cv2.COLOR_BGR2RGB))
  plt.show()

In [ ]:
img = 'desenho_sapato01'
exibir_resultados(img, diretorio_resultados)

In [ ]:
dir_imgs = 'imagens/test'
lista_imgs = [os.path.splitext(f)[0] for f in os.listdir(dir_imgs) if f.endswith(".jpg")]
lista_imgs

In [ ]:
for img in lista_imgs:
  print('\nResultado para a imagem {}'.format(img))
  exibir_resultados(img, diretorio_resultados)

# Dia para Noite

In [ ]:
nome_modelo = 'day2night'
diretorio_modelo = "./checkpoints/{}_pretreinado/".format(nome_modelo)
!mkdir -p {diretorio_modelo}

In [ ]:
url_modelo = "http://efrosgans.eecs.berkeley.edu/pix2pix/models-pytorch/{}.pth".format(nome_modelo)
!wget -N {url_modelo}

In [ ]:
!mv {nome_modelo}.pth {diretorio_modelo}latest_net_G.pth

In [ ]:
#Geração dos resultados

!python test.py --dataroot /content/teste --name {nome_modelo}_pretreinado --model test --dataset_mode single --netG unet_256 --norm batch